# 03. RAG (Retrieval-Augmented Generation) 시스템 구축

## 🎯 학습 목표
1. RAG의 개념과 필요성 이해
2. 임베딩과 벡터 검색 구현
3. 문서 처리 파이프라인 구축
4. 실전 RAG 시스템 개발

## 📚 RAG란 무엇인가?

## 1. RAG 시스템 아키텍처

### RAG의 핵심 개념

```
┌─────────────────────────────────────────────┐
│              RAG Pipeline                   │
├─────────────────────────────────────────────┤
│   1. 문서 수집     →    2. 청킹              │
│   (Documents)          (Chunking)           │
│                                             │
│   3. 임베딩 생성   →    4. 벡터 DB 저장      │
│   (Embedding)          (Vector Store)       │
│                                             │
│   5. 쿼리 임베딩   →    6. 유사도 검색       │
│   (Query Embed)        (Similarity Search)  │
│                                             │
│   7. 컨텍스트 생성  →   8. LLM 응답 생성     │
│   (Context)            (Generation)         │
└─────────────────────────────────────────────┘
```

### 왜 RAG가 필요한가?

1. **최신 정보**: LLM의 학습 데이터 시점 이후 정보 제공
2. **정확성**: 환각(Hallucination) 감소
3. **커스터마이징**: 도메인 특화 지식 활용
4. **출처 제공**: 답변의 근거 명시 가능
5. **비용 효율**: 파인튜닝 없이 지식 확장

## 2. 환경 설정

# NumPy 버전 충돌 해결 및 RAG 시스템 필수 패키지 설치
# 
# 주의: NumPy 2.x는 많은 머신러닝 패키지와 호환되지 않으므로 
# 1.x 버전으로 다운그레이드하고 호환되는 버전 조합으로 설치

# 1. 기존 패키지 제거 및 NumPy 다운그레이드
!pip uninstall numpy torch torchvision transformers sentence-transformers -y
!pip install numpy==1.24.3

# 2. 호환되는 버전 조합으로 재설치
!pip install torch==2.1.0 torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cpu
!pip install transformers==4.37.0 sentence-transformers==2.2.2

# 3. 추가 필요 패키지
!pip install chromadb langchain langchain-community ollama scikit-learn

print("✅ 패키지 설치 완료!")

In [1]:
import numpy as np
print(f"numpy {np.__version__} 정상 로드됨")

from sentence_transformers import SentenceTransformer
print("SentenceTransformer 정상 로드됨")

numpy 2.3.2 정상 로드됨
SentenceTransformer 정상 로드됨


In [2]:
# 패키지 버전 확인 및 호환성 검사
import warnings
warnings.filterwarnings('ignore')

def verify_installation():
    print("설치 확인:")
    
    try:
        # NumPy 확인
        import numpy
        print(f"✓ NumPy: {numpy.__version__}")
        
        # PyTorch 확인
        import torch
        print(f"✓ PyTorch: {torch.__version__}")
        
        # Torchvision 확인
        import torchvision
        print(f"✓ torchvision: {torchvision.__version__}")
        
        # Transformers 확인
        import transformers
        print(f"✓ transformers: {transformers.__version__}")
        
        # Sentence Transformers 테스트
        from sentence_transformers import SentenceTransformer
        print("✓ sentence-transformers 정상 로드됨")
        
        # 간단한 임베딩 테스트
        print("\n임베딩 모델 테스트 중...")
        model = SentenceTransformer('all-MiniLM-L6-v2')
        test_text = ["테스트 문장"]
        embedding = model.encode(test_text)
        print(f"✓ 임베딩 생성 성공: shape={embedding.shape}")
        
        return True
        
    except Exception as e:
        print(f"\n✗ 에러 발생: {e}")
        print("\n해결 방법:")
        print("1. 커널을 재시작하세요 (Kernel > Restart)")
        print("2. 위의 설치 셀을 다시 실행하세요")
        print("3. 문제가 지속되면 다음 명령어를 터미널에서 실행:")
        print("   pip uninstall numpy torch torchvision transformers sentence-transformers -y")
        print("   pip install numpy==1.24.3")
        print("   pip install torch==2.1.0 torchvision==0.16.0")
        print("   pip install transformers==4.37.0 sentence-transformers==2.2.2")
        return False

# 설치 확인 실행
if verify_installation():
    print("\n✅ 모든 패키지가 정상적으로 설치되었습니다!")
    print("이제 RAG 시스템을 구축할 준비가 완료되었습니다.")

설치 확인:
✓ NumPy: 2.3.2
✓ PyTorch: 2.8.0
✓ torchvision: 0.23.0
✓ transformers: 4.56.0
✓ sentence-transformers 정상 로드됨

임베딩 모델 테스트 중...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ 임베딩 생성 성공: shape=(1, 384)

✅ 모든 패키지가 정상적으로 설치되었습니다!
이제 RAG 시스템을 구축할 준비가 완료되었습니다.


In [3]:
# 패키지 버전 확인 및 호환성 문제 해결

def verify_installation():
    print("설치 확인:")
    
    try:
        import torch
        import torchvision
        import transformers
        from sentence_transformers import SentenceTransformer
        
        print("✓ 모든 패키지 정상 로드됨")
        print(f"PyTorch: {torch.__version__}")
        print(f"torchvision: {torchvision.__version__}")
        print(f"transformers: {transformers.__version__}")
        
        # 간단한 테스트
        model = SentenceTransformer('all-MiniLM-L6-v2')
        print("✓ SentenceTransformer 정상 작동")
        
    except Exception as e:
        print(f"✗ 에러 발생: {e}")

verify_installation()

설치 확인:
✓ 모든 패키지 정상 로드됨
PyTorch: 2.8.0
torchvision: 0.23.0
transformers: 4.56.0
✓ SentenceTransformer 정상 작동


In [ ]:
# 필요 패키지 설치
!pip install sentence-transformers chromadb langchain langchain-community -q

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from langchain_community.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
import json
from typing import List, Dict, Any

print("✅ 패키지 로드 완료")

✅ 패키지 로드 완료


## 3. 임베딩 기초

### 임베딩이란?
텍스트를 의미를 담은 벡터(숫자 배열)로 변환하는 과정입니다.

In [7]:
# 임베딩 모델 초기화
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# 텍스트 임베딩 생성
texts = [
    "Python은 프로그래밍 언어입니다",
    "파이썬은 코딩 언어입니다",
    "고양이는 동물입니다",
    "머신러닝은 AI의 한 분야입니다"
]

embeddings = embedder.encode(texts)

print(f"임베딩 차원: {embeddings.shape}")
print(f"첫 번째 텍스트의 임베딩 (처음 10개 값): {embeddings[0][:10]}")

임베딩 차원: (4, 384)
첫 번째 텍스트의 임베딩 (처음 10개 값): [-0.06128301  0.02552284 -0.02994079 -0.02846024 -0.04995072 -0.13498838
  0.03824809  0.03610769 -0.06114284 -0.05800613]


In [ ]:
# 유사도 계산
similarities = cosine_similarity(embeddings)

print("텍스트 간 유사도 매트릭스:")
for i, text1 in enumerate(texts):
    for j, text2 in enumerate(texts):
        if i < j:
            print(f"'{text1}' vs '{text2}': {similarities[i][j]:.3f}")

## 4. 벡터 데이터베이스 구축

ChromaDB를 사용하여 벡터 검색 시스템을 구축합니다.

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
import numpy as np

class VectorStore:
    def __init__(self, collection_name="knowledge_base"):
        # 새로운 ChromaDB 클라이언트
        self.client = chromadb.PersistentClient(path="./chroma_db")
        
        # 임베딩 모델
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        
        # 컬렉션 생성/로드
        try:
            self.collection = self.client.get_collection(collection_name)
            print(f"기존 컬렉션 '{collection_name}' 로드됨")
        except:
            self.collection = self.client.create_collection(collection_name)
            print(f"새 컬렉션 '{collection_name}' 생성됨")
    
    def add_documents(self, documents):
        """문서 추가"""
        embeddings = self.embedding_model.encode(documents)
        
        # ChromaDB에 추가
        self.collection.add(
            embeddings=embeddings.tolist(),
            documents=documents,
            ids=[f"doc_{i}" for i in range(len(documents))]
        )
        print(f"{len(documents)}개 문서 추가됨")
    
    def search(self, query, n_results=5):
        """유사도 검색"""
        query_embedding = self.embedding_model.encode([query])
        
        results = self.collection.query(
            query_embeddings=query_embedding.tolist(),
            n_results=n_results
        )
        return results

# 사용 예시
vector_store = VectorStore("rag_demo")

In [ ]:
# 지식 베이스 구축
knowledge_base = [
    "Python은 1991년 귀도 반 로섬이 개발한 고급 프로그래밍 언어입니다.",
    "Python은 간결하고 읽기 쉬운 문법으로 유명하며, 들여쓰기로 코드 블록을 구분합니다.",
    "머신러닝은 데이터에서 패턴을 학습하는 인공지능의 한 분야입니다.",
    "딥러닝은 인공 신경망을 사용하는 머신러닝의 하위 분야입니다.",
    "RAG는 Retrieval-Augmented Generation의 약자로, 검색 기반 생성 기법입니다.",
    "LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다.",
    "벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 시스템입니다.",
    "임베딩은 텍스트를 의미를 담은 벡터로 변환하는 과정입니다."
]

# 메타데이터 추가
# metadatas = [
#     {"topic": "Python", "category": "programming"},
#     {"topic": "Python", "category": "programming"},
#     {"topic": "ML", "category": "AI"},
#     {"topic": "DL", "category": "AI"},
#     {"topic": "RAG", "category": "AI"},
#     {"topic": "LangChain", "category": "tool"},
#     {"topic": "VectorDB", "category": "database"},
#     {"topic": "Embedding", "category": "AI"}
# ]

# 벡터 DB에 추가
# vector_store.add_documents(knowledge_base, metadatas)
vector_store.add_documents(knowledge_base)

In [ ]:
# 검색 테스트
queries = [
    "파이썬의 특징은?",
    "인공지능과 머신러닝의 관계",
    "RAG 시스템이란?"
]

for query in queries:
    print(f"\n🔍 Query: {query}")
    results = vector_store.search(query, n_results=2)
    
    for i, doc in enumerate(results['documents'][0]):
        distance = results['distances'][0][i] if results['distances'] else 0
        metadata = results['metadatas'][0][i] if results['metadatas'] else {}
        print(f"  [{i+1}] (거리: {distance:.3f})")
        print(f"      {doc[:80]}...")

## 5. 완전한 RAG 시스템 구현

In [ ]:
class RAGSystem:
    """완전한 RAG 시스템"""
    
    def __init__(self, llm_model="qwen3:8b", embedding_model="all-MiniLM-L6-v2"):
        # LLM 초기화
        self.llm = Ollama(model=llm_model)
        
        # 벡터 스토어
        self.vector_store = VectorStore("rag_system")
        
        # 텍스트 분할기
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50,
            separators=["\n\n", "\n", ".", " ", ""]
        )
    
    def add_document(self, text: str, source: str = "unknown"):
        """문서 추가 (자동 청킹)"""
        # 텍스트를 청크로 분할
        chunks = self.text_splitter.split_text(text)
        
        # 벡터 DB에 추가
        self.vector_store.add_documents(chunks)
        return len(chunks)
    
    def query(self, question: str, top_k: int = 3, use_thinking: bool = False):
        """RAG 기반 질의응답"""
        
        # 1. 관련 문서 검색
        search_results = self.vector_store.search(question, n_results=top_k)
        
        # 2. 컨텍스트 생성
        context_docs = search_results['documents'][0] if search_results['documents'] else []
        context = "\n\n".join(context_docs)
        
        # 3. 프롬프트 구성
        prompt = f"""
다음 컨텍스트를 참고하여 질문에 답변해주세요.
컨텍스트에 없는 내용은 추측하지 말고 "정보가 없습니다"라고 답하세요.

컨텍스트:
{context}

질문: {question}

답변:
"""
        
        # Thinking Mode 적용
        if use_thinking:
            prompt = f"/think {prompt}"
        
        # 4. LLM으로 응답 생성
        response = self.llm.invoke(prompt)
        
        return {
            "question": question,
            "answer": response,
            "sources": context_docs,
            "num_sources": len(context_docs)
        }

# RAG 시스템 초기화
rag = RAGSystem()
print("✅ RAG 시스템 준비 완료")

In [ ]:
# 문서 추가
documents = [
    """
    회사 규정 문서
    
    1. 근무 시간: 오전 9시 - 오후 6시 (점심시간 12시-1시)
    2. 재택근무: 주 2회 가능 (월/금 권장)
    3. 휴가: 연차 15일, 병가 10일
    4. 교육 지원: 연간 200만원 한도
    5. 회의: 매주 월요일 10시 팀 미팅
    """,
    
    """
    프로젝트 가이드라인
    
    1. 코드 리뷰: 모든 PR은 2명 이상의 리뷰 필요
    2. 테스트: 코드 커버리지 80% 이상 유지
    3. 문서화: 모든 공개 API는 문서화 필수
    4. 브랜치: feature/*, bugfix/*, hotfix/* 규칙 준수
    5. 배포: 매주 화요일, 목요일 정기 배포
    """,
    
    """
    기술 스택
    
    - 백엔드: Python (FastAPI), PostgreSQL
    - 프론트엔드: React, TypeScript, TailwindCSS
    - 인프라: AWS, Docker, Kubernetes
    - CI/CD: GitHub Actions, ArgoCD
    - 모니터링: Prometheus, Grafana, Sentry
    """
]

for i, doc in enumerate(documents):
    chunks = rag.add_document(doc, source=f"document_{i+1}")
    print(f"문서 {i+1}: {chunks}개 청크로 분할")

In [ ]:
# RAG 시스템 테스트
questions = [
    "재택근무는 언제 가능한가요?",
    "코드 리뷰 규칙은 무엇인가요?",
    "우리 회사는 어떤 프로그래밍 언어를 사용하나요?",
    "점심시간은 언제인가요?",
    "CEO는 누구인가요?"  # 컨텍스트에 없는 질문
]

for question in questions:
    print(f"\n{'='*60}")
    print(f"❓ 질문: {question}")
    
    result = rag.query(question)
    
    print(f"\n💡 답변: {result['answer']}")
    print(f"\n📚 참고한 소스 ({result['num_sources']}개):")
    for i, source in enumerate(result['sources'][:2]):
        print(f"  [{i+1}] {source[:100]}...")

## 6. 고급 RAG 기법

In [ ]:
class AdvancedRAG:
    """고급 RAG 기법 구현"""
    
    def __init__(self, llm_model="qwen3:8b"):
        self.llm = Ollama(model=llm_model)
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        self.documents = []
        self.embeddings = []
    
    def hybrid_search(self, query: str, top_k: int = 5):
        """하이브리드 검색 (의미 + 키워드)"""
        
        if not self.documents:
            return []
        
        # 1. 의미 기반 검색
        query_embedding = self.embedder.encode(query)
        semantic_scores = cosine_similarity([query_embedding], self.embeddings)[0]
        
        # 2. 키워드 기반 검색 (BM25 간단 구현)
        query_words = set(query.lower().split())
        keyword_scores = []
        
        for doc in self.documents:
            doc_words = set(doc.lower().split())
            overlap = len(query_words & doc_words)
            keyword_scores.append(overlap / max(len(query_words), 1))
        
        keyword_scores = np.array(keyword_scores)
        
        # 3. 점수 결합 (가중 평균)
        combined_scores = 0.7 * semantic_scores + 0.3 * keyword_scores
        
        # 4. 상위 k개 선택
        top_indices = np.argsort(combined_scores)[-top_k:][::-1]
        
        return [
            {
                "document": self.documents[idx],
                "score": combined_scores[idx],
                "semantic_score": semantic_scores[idx],
                "keyword_score": keyword_scores[idx]
            }
            for idx in top_indices
        ]
    
    def query_expansion(self, query: str) -> List[str]:
        """쿼리 확장 (관련 용어 추가)"""
        
        prompt = f"""
다음 질문과 관련된 유사 용어나 동의어를 3개 제시해주세요.
각 용어는 쉼표로 구분해주세요.

질문: {query}
유사 용어:
"""
        
        expansion = self.llm.invoke(prompt)
        expanded_terms = [term.strip() for term in expansion.split(',')]
        
        return [query] + expanded_terms[:3]
    
    def rerank_results(self, query: str, documents: List[str]) -> List[Dict]:
        """재순위 지정 (Cross-encoder 스타일)"""
        
        reranked = []
        
        for doc in documents:
            # 쿼리와 문서의 관련성을 LLM으로 평가
            prompt = f"""
다음 문서가 질문에 얼마나 관련이 있는지 0-10 점수로 평가해주세요.
숫자만 답하세요.

질문: {query}
문서: {doc[:200]}

점수:
"""
            
            try:
                score = float(self.llm.invoke(prompt).strip())
            except:
                score = 5.0
            
            reranked.append({
                "document": doc,
                "relevance_score": score
            })
        
        # 점수 기준 정렬
        reranked.sort(key=lambda x: x['relevance_score'], reverse=True)
        
        return reranked
    
    def add_documents(self, documents: List[str]):
        """문서 추가"""
        self.documents.extend(documents)
        new_embeddings = self.embedder.encode(documents)
        
        if len(self.embeddings) == 0:
            self.embeddings = new_embeddings
        else:
            self.embeddings = np.vstack([self.embeddings, new_embeddings])

# 고급 RAG 시스템 초기화
adv_rag = AdvancedRAG()

# 문서 추가
tech_docs = [
    "Python은 동적 타이핑을 지원하는 인터프리터 언어입니다.",
    "JavaScript는 웹 브라우저에서 실행되는 스크립트 언어입니다.",
    "Docker는 컨테이너화 기술을 제공하는 플랫폼입니다.",
    "Kubernetes는 컨테이너 오케스트레이션 도구입니다.",
    "Git은 분산 버전 관리 시스템입니다."
]

adv_rag.add_documents(tech_docs)

In [ ]:
# 하이브리드 검색 테스트
query = "컨테이너 관리 도구"

print("🔍 하이브리드 검색 결과:")
results = adv_rag.hybrid_search(query, top_k=3)

for i, result in enumerate(results, 1):
    print(f"\n[{i}] 종합 점수: {result['score']:.3f}")
    print(f"    의미 점수: {result['semantic_score']:.3f}")
    print(f"    키워드 점수: {result['keyword_score']:.3f}")
    print(f"    문서: {result['document']}")

In [ ]:
# 쿼리 확장 테스트
original_query = "프로그래밍 언어"

print(f"원본 쿼리: {original_query}")
expanded = adv_rag.query_expansion(original_query)
print(f"확장된 쿼리: {expanded}")

## 7. 실전 프로젝트: PDF 기반 Q&A 시스템

In [ ]:
class DocumentQA:
    """문서 기반 Q&A 시스템"""
    
    def __init__(self):
        self.rag = RAGSystem()
        self.sources = {}
    
    def load_text_file(self, filepath: str, source_name: str = None):
        """텍스트 파일 로드"""
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read()
            
            source = source_name or filepath
            chunks = self.rag.add_document(content, source)
            self.sources[source] = chunks
            
            print(f"✅ '{source}' 로드 완료 ({chunks} 청크)")
            return True
        except Exception as e:
            print(f"❌ 파일 로드 실패: {e}")
            return False
    
    def interactive_qa(self):
        """대화형 Q&A 세션"""
        print("\n💬 문서 Q&A 시스템")
        print("질문을 입력하세요 ('종료' 입력시 종료)\n")
        
        while True:
            question = input("질문: ")
            
            if question.lower() in ['종료', 'exit', 'quit']:
                print("👋 Q&A 세션 종료")
                break
            
            # 답변 생성
            result = self.rag.query(question)
            
            print(f"\n답변: {result['answer']}\n")
            print(f"(참고: {result['num_sources']}개 소스 활용)\n")
            print("-" * 50)
    
    def batch_qa(self, questions: List[str]) -> List[Dict]:
        """배치 Q&A 처리"""
        results = []
        
        for i, question in enumerate(questions, 1):
            print(f"처리중 [{i}/{len(questions)}]: {question[:50]}...")
            result = self.rag.query(question)
            results.append(result)
        
        return results

# 시스템 초기화
doc_qa = DocumentQA()

# 샘플 문서 생성
sample_doc = """
RAG 시스템 사용 가이드

1. 소개
RAG(Retrieval-Augmented Generation)는 검색과 생성을 결합한 AI 시스템입니다.
기존 LLM의 한계를 극복하고 최신 정보를 제공할 수 있습니다.

2. 주요 구성 요소
- 문서 처리: 텍스트를 작은 청크로 분할
- 임베딩: 텍스트를 벡터로 변환
- 벡터 DB: 임베딩을 저장하고 검색
- LLM: 컨텍스트 기반 응답 생성

3. 장점
- 환각 현상 감소
- 출처 제공 가능
- 도메인 특화 가능
- 실시간 정보 반영

4. 활용 분야
- 고객 서비스 챗봇
- 기술 문서 Q&A
- 법률 자문 시스템
- 의료 정보 검색
"""

# 임시 파일로 저장
with open('rag_guide.txt', 'w', encoding='utf-8') as f:
    f.write(sample_doc)

# 문서 로드
doc_qa.load_text_file('rag_guide.txt', 'RAG 가이드')

In [ ]:
# 배치 Q&A 테스트
test_questions = [
    "RAG의 주요 구성 요소는?",
    "RAG 시스템의 장점을 설명해주세요",
    "RAG는 어떤 분야에서 활용되나요?"
]

results = doc_qa.batch_qa(test_questions)

print("\n📊 배치 Q&A 결과:")
for i, (q, r) in enumerate(zip(test_questions, results), 1):
    print(f"\n[{i}] Q: {q}")
    print(f"    A: {r['answer'][:200]}...")

## 8. RAG 시스템 평가 및 개선

### 평가 메트릭
1. **검색 품질**: Precision, Recall, MRR
2. **생성 품질**: BLEU, ROUGE, 의미적 유사도
3. **전체 품질**: 정확도, 관련성, 완전성

### 개선 방법
1. **청킹 최적화**: 문서 특성에 맞는 청크 크기
2. **임베딩 모델 선택**: 도메인 특화 모델 사용
3. **하이브리드 검색**: 의미 + 키워드 검색 결합
4. **재순위**: Cross-encoder로 정확도 향상
5. **프롬프트 엔지니어링**: 더 나은 컨텍스트 활용

## 🎯 실습 과제

1. **기본 과제**:
   - 자신의 문서로 RAG 시스템 구축
   - 다양한 청크 크기 실험
   - 검색 결과 평가

2. **심화 과제**:
   - 멀티모달 RAG (이미지 + 텍스트)
   - 다국어 RAG 시스템
   - 실시간 업데이트 RAG

3. **프로젝트**:
   - 기술 문서 Q&A 봇
   - 논문 검색 및 요약 시스템
   - 코드베이스 지식 어시스턴트

## 📚 추가 학습 자료

- [RAG 논문](https://arxiv.org/abs/2005.11401)
- [ChromaDB 문서](https://docs.trychroma.com/)
- [Sentence Transformers](https://www.sbert.net/)
- [LlamaIndex RAG 가이드](https://gpt-index.readthedocs.io/)

## 다음 단계

다음 노트북에서는 **LoRA 파인튜닝**을 통해 모델을 커스터마이징하는 방법을 배워보겠습니다.